# IBM Applied Data Science Capstone Project

The Jupyter notebook will be used to complete the capstone project.

In [1]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Business Undertanding: Seattle Car Accident analyses 2004-2019

The City of Seattle through its Department of Transportation provides data free of charge to the public for all accidents and collisions since 2004. The purpose of this open data program has been to increase the safety of traffic, reduce accidents and increase the quality of living of its inhabitants. 

The aim of the analyses is to first understand accident hotspots and their circumstances, then predict the severity of accidents in the future so that the City of Seattle can act upon it to decrease accidents and injuries. We will focus on 2004 to 2019 as 2020 data are incomplete.

The data set is rich in details with own geocoding, street names, severity degrees, the weather, light and road condition when the accidents happened. As it covers nearly 200'000 accidents, focus needs to be laid in the analyses as the City of Seattle and its inhabitants benefit more from insights into hotspots and key reasons to change behaviour or infrastructure on Seattle streets.

In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import folium
from opencage.geocoder import OpenCageGeocode
import seaborn as sns
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')
from folium.plugins import MarkerCluster
import datetime

/bin/sh: conda: command not found
Folium installed and imported!


# DATA UNDERSTANDING

In [3]:
data = pd.read_csv ("/Users/Martin/Desktop/Data science capstone project Oct 2020/Coursera Seattle accident analyses/Data-Collisions.csv")

/Users/Martin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Downloading and Loading the Data: The table counts 194'673 accidents of different severity between 2003 and 2020, detailed in 37 + 1 index columns

In [4]:
data

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194668,2,-122.290826,47.565408,219543,309534,310814,E871089,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,24,From opposite direction - both moving - head-on,0,0,N
194669,1,-122.344526,47.690924,219544,309085,310365,E876731,Matched,Block,NaN,...,Wet,Daylight,NaN,NaN,NaN,13,From same direction - both going straight - bo...,0,0,N
194670,2,-122.306689,47.683047,219545,311280,312640,3809984,Matched,Intersection,24760.0,...,Dry,Daylight,NaN,NaN,NaN,28,From opposite direction - one left turn - one ...,0,0,N
194671,2,-122.355317,47.678734,219546,309514,310794,3810083,Matched,Intersection,24349.0,...,Dry,Dusk,NaN,NaN,NaN,5,Vehicle Strikes Pedalcyclist,4308,0,N


Columns with high figures of "0" (ie. "INATTENTIONID" has 164'868 blank fields) are not worth analysing. Although "Speeding" was named only in 9'333 cases as reasons for accidents, it will be analysed.
Internal Police codes are meaningless for this analysis. "SEVERITYDESC" is a verbal repetition of SEVERITYCODE 1 or 2. After deep thought, we decided to drop PEDCOUNT and PEDCYLCOUNTAll to focus on original accidents and the available SEVERITYCODE 1 and 2 descriptions for every accidents. 

In [36]:
drop_these =['OBJECTID', "INCKEY","COLDETKEY","REPORTNO","STATUS","ADDRTYPE",
             "INTKEY","EXCEPTRSNCODE","EXCEPTRSNDESC","SEVERITYCODE.1","SEVERITYDESC","COLLISIONTYPE","PERSONCOUNT",
             "JUNCTIONTYPE","SDOT_COLCODE","SDOT_COLDESC","INATTENTIONIND","PEDCOUNT","PEDCYLCOUNT","PEDROWNOTGRNT","SDOTCOLNUM","ST_COLCODE","ST_COLDESC","SEGLANEKEY","CROSSWALKKEY","HITPARKEDCAR"]
core_data=data.drop(columns=drop_these)
core_data

,SEVERITYCODE,X,Y,LOCATION,VEHCOUNT,INCDATE,INCDTTM,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,-122.323148,47.703140,5TH AVE NE AND NE 103RD ST,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,N,Overcast,Wet,Daylight,NaN
1,1,-122.347294,47.647172,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,-122.334540,47.607871,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,0,Overcast,Dry,Daylight,NaN
3,1,-122.334803,47.604803,2ND AVE BETWEEN MARION ST AND MADISON ST,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,N,Clear,Dry,Daylight,NaN
4,2,-122.306426,47.545739,SWIFT AVE S AND SWIFT AV OFF RP,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,0,Raining,Wet,Daylight,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
194668,2,-122.290826,47.565408,34TH AVE S BETWEEN S DAKOTA ST AND S GENESEE ST,2,2018/11/12 00:00:00+00,11/12/2018 8:12:00 AM,N,Clear,Dry,Daylight,NaN
194669,1,-122.344526,47.690924,AURORA AVE N BETWEEN N 85TH ST AND N 86TH ST,2,2018/12/18 00:00:00+00,12/18/2018 9:14:00 AM,N,Raining,Wet,Daylight,NaN
194670,2,-122.306689,47.683047,20TH AVE NE AND NE 75TH ST,2,2019/01/19 00:00:00+00,1/19/2019 9:25:00 AM,N,Clear,Dry,Daylight,NaN
194671,2,-122.355317,47.678734,GREENWOOD AVE N AND N 68TH ST,1,2019/01/15 00:00:00+00,1/15/2019 4:48:00 PM,N,Clear,Dry,Dusk,NaN


26 columns have been dropped to allow a more focussed and less memory heavy data analysis

In [6]:
core_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   SEVERITYCODE  194673 non-null  int64  
 1   X             189339 non-null  float64
 2   Y             189339 non-null  float64
 3   LOCATION      191996 non-null  object 
 4   VEHCOUNT      194673 non-null  int64  
 5   INCDATE       194673 non-null  object 
 6   INCDTTM       194673 non-null  object 
 7   UNDERINFL     189789 non-null  object 
 8   WEATHER       189592 non-null  object 
 9   ROADCOND      189661 non-null  object 
 10  LIGHTCOND     189503 non-null  object 
 11  SPEEDING      9333 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 17.8+ MB


In [7]:
core_data['SEVERITYCODE'].value_counts().to_frame('severitycount')

,severitycount
1,136485
2,58188


1=injury; 2= property damage
nearly twice as many injuries as property damages occured. For AI / ML purposes, the number of 1 and 2 will be balanced to a total of 58188 each in a later step.

In [8]:
core_data["VEHCOUNT"].value_counts().to_frame('Nr_vehicles')

,Nr_vehicles
2,147650
1,25748
3,13010
0,5085
4,2426
5,529
6,146
7,46
8,15
9,9


In most reported accidents, 2 vehicles were involved

In [9]:
core_data['ROADCOND'].value_counts().to_frame('Nr_accidents_by_Road_condition')

,Nr_accidents_by_Road_condition
Dry,124510
Wet,47474
Unknown,15078
Ice,1209
Snow/Slush,1004
Other,132
Standing Water,115
Sand/Mud/Dirt,75
Oil,64


Surprisingly, most accidents happened on dry roads. Unfortunatly, in 15'078 cases no road condition was recorded.

In [10]:
core_data['WEATHER'].value_counts().to_frame('weathertypes')

,weathertypes
Clear,111135
Raining,33145
Overcast,27714
Unknown,15091
Snowing,907
Other,832
Fog/Smog/Smoke,569
Sleet/Hail/Freezing Rain,113
Blowing Sand/Dirt,56
Severe Crosswind,25


111'135 accidents happended in clear weather conditions. Unfortunately, in 15'091 cases no weather data was recorded.

In [11]:
core_data['LIGHTCOND'].value_counts().to_frame('light_conditions')

,light_conditions
Daylight,116137
Dark - Street Lights On,48507
Unknown,13473
Dusk,5902
Dawn,2502
Dark - No Street Lights,1537
Dark - Street Lights Off,1199
Other,235
Dark - Unknown Lighting,11


116'137 incidents happened during daylight.

In [12]:
core_data['SPEEDING'].value_counts().to_frame('speeding_count')

,speeding_count
Y,9333


Speeding played a minor role in accidents with solely 9'333 cases.

In [13]:
core_data['LOCATION'].value_counts().to_frame('count').head(15)

,count
BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N,276
BATTERY ST TUNNEL SB BETWEEN AURORA AVE N AND ALASKAN WY VI SB,271
N NORTHGATE WAY BETWEEN MERIDIAN AVE N AND CORLISS AVE N,265
AURORA AVE N BETWEEN N 117TH PL AND N 125TH ST,254
6TH AVE AND JAMES ST,252
AURORA AVE N BETWEEN N 130TH ST AND N 135TH ST,239
ALASKAN WY VI NB BETWEEN S ROYAL BROUGHAM WAY ON RP AND SENECA ST OFF RP,238
RAINIER AVE S BETWEEN S BAYVIEW ST AND S MCCLELLAN ST,231
ALASKAN WY VI SB BETWEEN COLUMBIA ST ON RP AND ALASKAN WY VI SB EFR OFF RP,212
WEST SEATTLE BR EB BETWEEN ALASKAN WY VI NB ON RP AND DELRIDGE-W SEATTLE BR EB ON RP,212


There are clear "hot spots" where accidents happen, ie Battery St Tunnel, N Northgate Way, Aurora Ave, etc. This is demonstrated with the following graph overlaying the accident locations with the Seattle map.

In [38]:
map = folium.Map(location=[47.60, -122.33], zoom_start=13)
marker_cluster = MarkerCluster().add_to(map)
locations = core_data[['Y', 'X']][core_data['Y'].notna()].head(500)
locationlist = locations.values.tolist()
for point in range(len(locations)):
    folium.Marker(locationlist[point]).add_to(marker_cluster)
map.save('mymap.html')
map

In [15]:
Number_different_spots=core_data["Y"].value_counts()
spots=Number_different_spots

In [16]:
figure_spots=spots.to_frame(name='Number_of_spots_by_latitude')
figure_spots

,Number_of_spots_by_latitude
47.708655,265
47.717173,254
47.604161,252
47.725036,239
47.579673,231
...,...
47.556705,1
47.709101,1
47.513899,1
47.565438,1


The total of 193'000 incidents where spread across 23'839 spots in Seattle taking latitude data. There were 53 key spots with >= 100 incidents per degree of latitude across Seattle in the analysed timeframe.

In [17]:
new=figure_spots[figure_spots.Number_of_spots_by_latitude >= 100]
new.count()

Number_of_spots_by_latitude    53
dtype: int64

In [18]:
core_data['UNDERINFL'].value_counts().to_frame('count')

,count
N,100274
0,80394
Y,5126
1,3995


#### In week 3, analyses for this Capstone project the data preparation and cleaning will be key. Categorical values will need to be changed into figures. Some descriptions will be amalgamated into one summed up row description. NaN and blank cells/rows will be deleted.